In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D

import tensorflow.keras.backend as K
import tensorflow as tf

import cv2
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import random
import os
import tensorflow as tf
import keras as keras
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
import src.proprietary_functions as src

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
from tensorflow.keras.datasets import mnist

In [2]:
#list of samples strings
list_samples = ['train','valid','test']
#dictionary for storing cropped images, image names and labels
arr_dicts = {}

In [3]:
#importing numpy objects of cropped images
for sampl in list_samples:
    with open(f'./cropped_numpys/cropped_{sampl}_X.npy', 'rb') as f:
        arr_dicts[f'arr_{sampl}_X']  = np.load(f)

arr_train_X = arr_dicts['arr_train_X']
arr_valid_X = arr_dicts['arr_valid_X']
arr_test_X = arr_dicts['arr_test_X']

In [4]:
#importing numpy objects of image names
for sampl in list_samples:
    with open(f'./cropped_numpys/cropped_{sampl}_X_names.npy', 'rb') as f:
        arr_dicts[f'{sampl}_X_names']  = np.load(f)

train_X_names = arr_dicts['train_X_names']
valid_X_names = arr_dicts['valid_X_names']
test_X_names = arr_dicts['test_X_names']

In [5]:
#importing numpy objects of labels
for sampl in list_samples:
    with open(f'./cropped_numpys/cropped_{sampl}_Y.npy', 'rb') as f:
        arr_dicts[f'arr_{sampl}_Y']  = np.load(f)

arr_train_Y = arr_dicts['arr_train_Y']
arr_valid_Y = arr_dicts['arr_valid_Y']
arr_test_Y = arr_dicts['arr_test_Y']

In [6]:
#function for creation of triplets
def	make_triplets(images, labels, image_names):

	tripletImages = []
	tripletImagesNames = []
	uniqueClasses = np.unique(labels)

	dict_idx = {i:np.where(labels == i)[0] for i in uniqueClasses}

	for idxA in range(len(images)):

		#current image
		currentImage = images[idxA]
		label = labels[idxA]
		currentImage_name = image_names[idxA]

		#positive image
		idxB = np.random.choice(dict_idx[label])
		posImage = images[idxB]
		posImage_name = image_names[idxB]

		#negative image
		negLab = np.random.choice([i for i in dict_idx.keys() if i != label])
		negIdx = np.random.choice(dict_idx[negLab])
		negImage = images[negIdx]
		negImage_name = image_names[negIdx]
        
		#saving the triplets of images and image names
		tripletImages.append([currentImage, posImage, negImage])
		tripletImagesNames.append([currentImage_name, posImage_name, negImage_name])

	return (np.array(tripletImages), np.array(tripletImagesNames))

In [7]:
train_X_triplets, train_triplets_names= make_triplets(arr_train_X, arr_train_Y, train_X_names)
valid_X_triplets, valid_triplets_names = make_triplets(arr_valid_X, arr_valid_Y, valid_X_names)
test_X_triplets, test_triplets_names = make_triplets(arr_test_X, arr_test_Y, test_X_names)

In [8]:
if not os.path.exists('./triplet_numpys/'):
    os.makedirs('./triplet_numpys/')

In [9]:
dict_triplets = {'train': {'triplet_imgs': train_X_triplets, 'triplet_imgs_names':train_triplets_names},
                'valid': {'triplet_imgs': valid_X_triplets,'triplet_imgs_names':valid_triplets_names},
                'test': {'triplet_imgs': test_X_triplets, 'triplet_imgs_names':test_triplets_names}}

In [10]:
#exporting the triplets of images and image names as numpy objects.
for sampl in dict_triplets.keys():
    for n in dict_triplets[sampl].keys():
       with open(f'./triplet_numpys/{sampl}_{n}.npy', 'wb') as f:
            np.save(f, dict_triplets[sampl][n])

NN model building with triplet loss

In [11]:
def initialize_base_network():
    input = Input(shape=(224,224,3))
    x = Flatten()(input)
    x = Dense(120, activation='relu')(x)
    return Model(inputs=input, outputs=x)

embedding = initialize_base_network()
tf.keras.utils.plot_model(embedding, show_shapes=True)

2022-12-13 13:26:16.672090: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [12]:
class SiameseNet(tf.keras.layers.Layer):
    # set the backbone model in constructor
    def __init__(self, model):
        super().__init__()
        self.model = model

    def call(self, feat):
        # get feature vectors from anchor
        feats = self.model(feat[0])
        # from positive image
        pfeats = self.model(feat[1])
        # and from negative image
        nfeats = self.model(feat[2])
        # concatenate vectors to a matrix
        result = tf.stack([feats, pfeats, nfeats])
        return result

class TripletLoss(tf.keras.layers.Layer):
    # margin is settable hyperparameter in constructor
    def __init__(self, margin):
        self.margin = margin
        super().__init__()
        
    # function calculating distance between features
    def distance(self, x, y):
        sum_square = tf.reduce_sum(tf.square(x - y), axis=1, keepdims=True)
        return tf.sqrt(tf.maximum(sum_square, tf.keras.backend.epsilon()))

    def call(self, features):
        # get anchor-positive distance
        pos = self.distance(features[0], features[1])
        # anchor-negative distance
        neg = self.distance(features[0], features[2])
        # difference between anchor positive and anchor negative distances
        loss = pos - neg
        # get overall loss
        return tf.maximum(loss + self.margin, 0.0)

In [13]:
# anchor branch
image_input = Input(shape=(224,224,3), name='image_input')
# positive image branch
positive_input = Input(shape=(224,224,3), name='positive_input')
# negative image branch
negative_input = Input(shape=(224,224,3), name='negative_input')

In [14]:
def identity_loss(y_true, y_pred):
    return tf.reduce_mean(abs(y_true-y_pred))

In [15]:
siamese = SiameseNet(embedding)([image_input, positive_input, negative_input])
loss = TripletLoss(margin=1.0)(siamese)
model = Model(inputs=[image_input, positive_input, negative_input], outputs=loss)
model.compile(optimizer ='adam', loss = identity_loss)

In [16]:
history = model.fit([train_X_triplets[:,0], train_X_triplets[:,1], train_X_triplets[:,2]], np.ones(train_X_triplets.shape[0]),
                verbose=1,
            validation_data=([valid_X_triplets[:,0], valid_X_triplets[:,1], valid_X_triplets[:,2]], np.ones(valid_X_triplets.shape[0])),
            epochs=10)
            #epochs = 2)

Epoch 1/10
188/188 [==============================] - 33s 164ms/step - loss: 256.0246 - val_loss: 1.5270
Epoch 2/10
188/188 [==============================] - 24s 126ms/step - loss: 5.0000e-04 - val_loss: 1.5270
Epoch 3/10
188/188 [==============================] - 26s 135ms/step - loss: 5.0000e-04 - val_loss: 1.5270
Epoch 4/10
188/188 [==============================] - 28s 150ms/step - loss: 5.0000e-04 - val_loss: 1.5270
Epoch 5/10
188/188 [==============================] - 28s 145ms/step - loss: 5.0000e-04 - val_loss: 1.5270
Epoch 6/10
188/188 [==============================] - 27s 143ms/step - loss: 5.0000e-04 - val_loss: 1.5270
Epoch 7/10
188/188 [==============================] - 30s 158ms/step - loss: 5.0000e-04 - val_loss: 1.5270
Epoch 8/10
188/188 [==============================] - 29s 155ms/step - loss: 5.0000e-04 - val_loss: 1.5270
Epoch 9/10
188/188 [==============================] - 27s 141ms/step - loss: 5.0000e-04 - val_loss: 1.5270
Epoch 10/10
188/188 [==================

Predictions

In [23]:
import pickle

model_file = "./triplets.sav"
pickle.dump(model, open(model_file, 'wb'))

INFO:tensorflow:Assets written to: ram://a3093573-a704-4d2d-9e39-b2f2ba2f80db/assets


In [17]:
#function creating pairs
def	make_pairs(images, labels):

	pairImages = []
	pairLabels = []
	uniqueClasses = np.unique(labels)

	dict_idx = {i:np.where(labels == i)[0] for i in uniqueClasses}

	for idxA in range(len(images)):
		currentImage = images[idxA]
		label = labels[idxA]

		#positive pair
		idxB = np.random.choice(dict_idx[label])
		posImage = images[idxB]
		pairImages.append([currentImage, posImage])

		pairLabels.append([1])

		#negative pair
		negLab = np.random.choice([i for i in dict_idx.keys() if i != label])
		negIdx = np.random.choice(dict_idx[negLab])
		negImage = images[negIdx]
		pairImages.append([currentImage, negImage])

		pairLabels.append([0])

	return (np.array(pairImages),np.array(pairLabels))

In [18]:
pairTest, labelTest = make_pairs(arr_test_X, arr_test_Y)

In [19]:
left_pair = pairTest[:,0]
right_pair = pairTest[:,1]


In [20]:
left_pair_pred = embedding.predict(left_pair)
right_pair_pred = embedding.predict(right_pair)

125/125 [==============================] - 2s 18ms/step


In [21]:
def compute_accuracy(left_pred, right_pred, y_true):
    y_pred = np.linalg.norm(left_pred - right_pred, axis=1)
#     # 1 for the same - distance is smaller than 3.0, 0 for the different
    pred = y_pred < 7.0
    return np.mean(pred == y_true)

In [22]:
test_accuracy = compute_accuracy(left_pair_pred, right_pair_pred, labelTest)
print(f'Test accuracy: {test_accuracy*100:.2f}%')

Test accuracy: 50.00%
